In [1]:
from process import ProcessRNRDataset
threadId, postId, threadTag, structures, posts = ProcessRNRDataset()

charliehebdo rumor: 100%|██████████| 459/459 [00:00<00:00, 775.00it/s]
ebola-essien non-rumor: 0it [00:00, ?it/s]
sydneysiege rumor: 100%|██████████| 523/523 [00:00<00:00, 750.97it/s]


In [ ]:
with open('./all/thread_id.txt', 'w') as f:
    for id in threadId:
        f.write(id + '\n')


In [4]:
with open('./all/post_id.txt', 'w') as f:
    for id in postId:
        f.write(id + '\n')
        

In [5]:
with open('./all/thread_label.txt', 'w') as f:
    for id in threadId:
        f.write(str(threadTag[id]) + '\n')

In [6]:
import json
with open('./all/structures.json', 'w') as f:
    json.dump(structures, f)

In [7]:
import json
with open('./all/posts.json', 'w') as f:
    json.dump(posts, f)

In [ ]:
import json
from process import label2Index
with open('./rumorCategory.json', 'w') as f:
    json.dump(label2Index, f)

In [2]:
print(len(postId))
print(len(posts))

104582
104582


In [5]:
for id in postId:
    try:
        a = posts[id]
    except Exception as e:
        print(e)

In [ ]:
import json
with open('./all/posts.json', 'w') as f:
    json.dump(posts, f)

### 补充的PHEME立场标签

In [5]:
import os
import pandas as pd
import json

with open('./stanceCategory.json', 'r') as f:
    stance2label = json.load(f)
with open('./all/thread_id.txt', 'r') as f:
    thread_ids = [id.strip() for id in f.readlines()]

In [7]:
len(thread_ids)

6425

In [8]:
df = pd.read_excel('./all/all-PHEME-stance-semeval-set.xlsx')
df['id'] = df['id'].astype(str)

In [11]:
with open('./all/post_label.txt', 'w') as fout:
    stances = {row['id']:row['stance'] for _, row in df.iterrows()}
    posts = []
    with open('./all/post_id.txt', 'r') as fposts:
        posts += [post.strip() for post in fposts.readlines()]
    for post in posts:
        if post not in stances:
            fout.write(str(stance2label['comment']) + '\n')
        elif post in thread_ids:
            fout.write(str(stance2label['support']) + '\n')
        else:
            fout.write(str(stance2label[stances[post]]) + '\n')

### 统计各个事件的立场标签情况

In [12]:
from process import label2Index, index2Label, DATA_PATH

posts = []
stances = []

with open('./all/post_id.txt', 'r') as f:
    posts += [post.strip() for post in f.readlines()]
with open('./all/post_label.txt', 'r') as f:
    stances += [int(stance.strip()) for stance in f.readlines()]

post_id2stance = {post:stance for post, stance in zip(posts, stances)}


In [13]:
import os
eventName = os.listdir(DATA_PATH)
print(eventName)

['charliehebdo', 'ebola-essien', 'ferguson', 'germanwings-crash', 'gurlitt', 'ottawashooting', 'prince-toronto', 'putinmissing', 'sydneysiege', '.DS_Store']


In [14]:
from tqdm import tqdm
import json
from process import strTime2Timestamp
from convert_veracity_annotations import convert_annotations
import numpy as np

for event in eventName:
    if '.' in event: continue
    print('eventName: ', event)
    thread_id = []
    post_id = []
    thread_tag = {}
    structures = {}
    posts = {}

    ids = os.listdir(os.path.join(DATA_PATH, event, 'non-rumours'))
    for tid in ids:
        if '.' in tid: continue
        path = os.path.join(DATA_PATH, event, 'non-rumours', str(tid))

        # source tweet id
        thread_id.append(tid)
        post_id.append(tid)

        # thread tag(non-rumor)
        thread_tag[tid] = 0

        # structure
        with open(os.path.join(path, 'structure.json'), 'r') as f:
            content = f.read()
        structures[tid] = json.loads(content)

        pids = os.listdir(os.path.join(path, 'reactions'))
        for pfname in pids:
            if '._' not in pfname and '.json' in pfname:
                pid = pfname[0:-5]
                post_id.append(pid)


    ids = os.listdir(os.path.join(DATA_PATH, event, 'rumours'))
    for tid in ids:
        if '.' in tid: continue
        path = os.path.join(DATA_PATH, event, 'rumours', str(tid))
        # id
        thread_id.append(tid)
        post_id.append(tid)
        # tag
        with open(os.path.join(path, 'annotation.json'), 'r') as f:
            annotation = json.loads(f.read())
            label = convert_annotations(annotation)
        thread_tag[tid] = label2Index[label]
        # structure
        with open(os.path.join(path, 'structure.json'), 'r') as f:
            content = f.read()
        structures[tid] = json.loads(content)

        pids = os.listdir(os.path.join(path, 'reactions'))
        for pfname in pids:
            if '._' not in pfname and '.json' in pfname:
                pid = pfname[0:-5]
                post_id.append(pid)      
    
    print('会话数量: ', len(thread_id))
    print('推文数量: ', len(post_id))
    thread_tag = np.array([int(thread_tag[id]) for id in thread_tag])
    print('谣言标签：T: true-rumor:', (thread_tag == 1).sum(), ', false-rumor:', (thread_tag == 2).sum(),\
          ', unverified-rumor', (thread_tag == 3).sum(), ', F:', (thread_tag == 0).sum())
    stances = np.array([int(post_id2stance[post]) for post in post_id])
    print('立场标签: S:', (stances == 0).sum(), ', Q:', (stances == 1).sum(),\
          ', D:', (stances == 2).sum(), ', C:', (stances == 3).sum())

eventName:  charliehebdo
会话数量:  2079
推文数量:  38268
谣言标签：T: true-rumor: 193 , false-rumor: 116 , unverified-rumor 149 , F: 1621
立场标签: S: 3296 , Q: 1188 , D: 1117 , C: 32667
eventName:  ebola-essien
会话数量:  14
推文数量:  226
谣言标签：T: true-rumor: 0 , false-rumor: 14 , unverified-rumor 0 , F: 0
立场标签: S: 20 , Q: 2 , D: 29 , C: 175
eventName:  ferguson
会话数量:  1143
推文数量:  24175
谣言标签：T: true-rumor: 10 , false-rumor: 8 , unverified-rumor 266 , F: 859
立场标签: S: 2431 , Q: 1121 , D: 888 , C: 19735
eventName:  germanwings-crash
会话数量:  469
推文数量:  4489
谣言标签：T: true-rumor: 94 , false-rumor: 111 , unverified-rumor 33 , F: 231
立场标签: S: 746 , Q: 279 , D: 137 , C: 3327
eventName:  gurlitt
会话数量:  138
推文数量:  179
谣言标签：T: true-rumor: 59 , false-rumor: 0 , unverified-rumor 2 , F: 77
立场标签: S: 141 , Q: 1 , D: 0 , C: 37
eventName:  ottawashooting
会话数量:  890
推文数量:  12284
谣言标签：T: true-rumor: 329 , false-rumor: 72 , unverified-rumor 69 , F: 420
立场标签: S: 1302 , Q: 576 , D: 373 , C: 10033
eventName:  prince-toronto
会话数量:  233